
Notebook criado por: Lorena M. Santos. Você pode salvar uma cópia se quiser, clicando em File --> Save a Copy in Drive.

# Criando um Classificador de Sentimentos Simples usando TensorFlow

Neste exemplo, criamos um classificador de sentimentos básico usando TensorFlow. O modelo recebe frases de entrada e determina se elas têm um sentimento positivo ou negativo, utilizando uma abordagem simples de embedding e redes neurais densas.

## O Que Este Modelo Faz?
O modelo aqui desenvolvido converte as palavras em vetores numéricos (usando a camada Embedding).

Com base nesses vetores, o modelo aprende a identificar padrões associados a frases positivas ou negativas.

Ele prevê um valor entre 0 e 1:
- Valores acima de 0.5 são considerados positivos.
- Valores abaixo de 0.5 são considerados negativos.

Esse modelo é **simples** e foi treinado do zero em um pequeno conjunto de dados. **Ele serve como uma introdução à construção de modelos de NLP** e pode ser expandido para conjuntos de dados maiores e mais complexos.

O intuito é demonstrar um pouco como o TensorFlow é utilizado em tarefas de deep learnig, mostrando suas bibliotecas, e como este framework facilita a implementação de códigos relacionados à uma tarefa de NLP, por exemplo.

### Primeiro Passo: Importando Biblliotecas

- TensorFlow: Biblioteca de aprendizado de máquina amplamente utilizada para deep learning.
- Keras: API de alto nível do TensorFlow para criação de modelos.
- Tokenizer: Converte textos em sequências de números.
- pad_sequences: Garante que todas as sequências tenham o mesmo tamanho, preenchendo com zeros quando necessário.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Criando as Sentenças e Labels

Aqui, criamos:

- Sentenças: Uma lista simples de frases para treino (positivo/negativo).
- Labels: Rótulos binários representando o sentimento (1 para positivo, 0 para negativo).

In [ ]:
# 1. Dados simples
sentences = [
    "Eu amo programação",
    "Este filme é terrível",
    "O dia está lindo",
    "Eu detesto esperar",
    "Que lugar maravilhoso!",
    "Isso é uma perda de tempo"
]
labels = [1, 0, 1, 0, 1, 0]  # 1 = positivo, 0 = negativo

### Preprocessamento dos Dados

Para o pre-processamento, temos:

- Tokenizer: Converte cada palavra em um número único (token).
- `num_words=1000:` Considera as 1000 palavras mais comuns.
- `oov_token="<OOV>"`: Define um token para palavras fora do vocabulário.
- `texts_to_sequences`: Transforma as frases em listas de tokens.
- `pad_sequences`: Preenche as listas para que todas tenham o mesmo comprimento.

In [ ]:
# 2. Pré-processamento
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, padding="post")

### Criando a Arquitetura do Modelo

Na criação do modelo vamos usar

1. **Sequential:** Um modelo linear de camadas.
O Sequential é a forma mais simples de criar um modelo no Keras/TensorFlow.
Ele organiza as camadas de forma linear, ou seja, uma camada é conectada diretamente à próxima, como uma "pilha" de camadas. É ideal para problemas simples, onde os dados fluem de maneira ordenada entre as camadas, sem bifurcações ou junções complexas.

2. **Embedding:** Camada que cria representações densas (embeddings) para as palavras. A camada Embedding converte palavras em vetores numéricos de tamanho fixo, chamados embeddings. Cada palavra é transformada em um vetor de números contínuos que representam características da palavra.

3. `input_dim=1000`: Número máximo de palavras no vocabulário.
Define o tamanho do vocabulário, ou seja, o número máximo de palavras únicas que o modelo consegue aprender. Se houver mais palavras, elas serão representadas pelo token <OOV> (Out Of Vocabulary).

4. `output_dim=16`: Tamanho do vetor de cada palavra.
É o tamanho dos vetores gerados para cada palavra. Neste caso, cada palavra é representada como um vetor de 16 números.
Esse valor pode ser ajustado; tamanhos maiores capturam mais informações, mas tornam o modelo mais lento.

5. `input_length`:
É o comprimento máximo das sequências de entrada, usado para garantir compatibilidade entre os dados e a camada.

6. `GlobalAveragePooling1D`: Reduz os embeddings a um vetor único.
Após a camada Embedding, temos uma sequência de vetores (um vetor por palavra).
O GlobalAveragePooling1D pega todos esses vetores e calcula a média deles, criando um único vetor final.
Isso simplifica os dados e reduz a dimensão da entrada para as próximas camadas.

  **Por que isso é útil?** Ele mantém uma representação global do texto (média das palavras), removendo a dependência do comprimento da frase.


7.  Camada `Dense`:

  a. 16 neurônios (ReLU): Camada oculta para aprendizado. A camada Dense conecta todos os neurônios da entrada com todos os neurônios da saída.

  *   16 neurônios: Define o número de unidades (ou neurônios) nesta camada. Quanto mais neurônios, mais complexa será a representação aprendida.

  *   Ativação ReLU: A função ReLU (Rectified Linear Unit) transforma as saídas negativas em zero e mantém as positivas. É amplamente utilizada em redes neurais por ser eficiente e reduzir o risco de saturação do modelo.A camada Dense conecta todos os neurônios da entrada com todos os neurônios da saída.
  
  b. neurônio (Sigmoid): Camada de saída que prevê probabilidade binária (positivo/negativo). Esta é a camada de saída do modelo.

  * 1 neurônio signfica que a saída é um único valor, que será entre 0 e 1, devido à ativação da sigmoid.

  * Ativação Sigmoid: A função sigmoide transforma a saída em uma probabilidade (número entre 0 e 1). Ela é ideal para problemas de classificação binária, onde o objetivo é prever "positivo" ou "negativo".


8. Compilação do Modelo:

  a. Otimizador: Adam
  - O Adam (Adaptive Moment Estimation) é um dos otimizadores mais eficientes e populares em deep learning.
  - Ele ajusta os pesos do modelo de forma adaptativa, acelerando o aprendizado e evitando que o modelo fique preso em mínimos locais.

  b. Função de perda: Binary Crossentropy
  - Binary Crossentropy mede a diferença entre as previsões do modelo e os rótulos reais.
  - É a função ideal para problemas de classificação binária, penalizando previsões incorretas mais severamente.

  c. Métrica: Acurácia
  - A acurácia mede o percentual de previsões corretas feitas pelo modelo.
  - É uma métrica intuitiva para entender o desempenho do modelo em tarefas de classificação binária.

In [ ]:
# 3. Criação do modelo
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=padded_sequences.shape[1]),
    GlobalAveragePooling1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



### Treinamento do Modelo

- Conversão dos dados: Garante que as sequências e rótulos estejam no formato correto (float32).
- Treinamento:
  - epochs=10: O modelo passa pelos dados 10 vezes.
  - verbose=0: Supressão da saída de progresso para visualização mais limpa.


In [ ]:
# 4. Treinamento
# Convert padded_sequences and labels to NumPy arrays with dtype=float32
padded_sequences = padded_sequences.astype('float32')
labels = tf.convert_to_tensor(labels, dtype=tf.float32) # or labels = np.array(labels, dtype=np.float32)

model.fit(padded_sequences, labels, epochs=10, verbose=0)


O model.summary() é uma função do Keras que fornece um resumo da arquitetura do seu modelo. Com ela obtemos uma representação visual da estrutura do classificador de sentimentos, mostrando como as camadas estão conectadas, seus parâmetros e o formato dos dados à medida que passam pelo modelo.

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 6, 16)               │          16,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 16)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 48,869 (190.90 KB)

 Trainable params: 16,289 (63.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 32,580 (127.27 KB)

### Testando o Modelo e Resultados

- Entrada de teste: Uma nova frase para prever o sentimento.
- `texts_to_sequences`: Transforma a frase em tokens usando o mesmo tokenizer treinado.
- `pad_sequences`: Ajusta o tamanho da sequência para o mesmo valor usado durante o treinamento.

- Predição:
 - O modelo retorna um valor entre 0 e 1.
 - Se > 0.5: Considera positivo se a probabilidade for maior que 50%.

- Resultado: O sentimento é exibido como "Positivo" ou "Negativo".

In [ ]:
# 5. Testando o modelo
test_sentence = ["Eu amo pizza!"]
test_seq = tokenizer.texts_to_sequences(test_sentence)
test_pad = pad_sequences(test_seq, maxlen=padded_sequences.shape[1])
# Convert test_pad to float32
test_pad = test_pad.astype('float32')
prediction = model.predict(test_pad)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [ ]:
# imprimindo prediction para conferência
prediction

0.5001468

In [ ]:
print(f"Sentimento: {'Positivo' if prediction > 0.50 else 'Negativo'}")

Sentimento: Positivo


## Resultado Final

O modelo consegue prever o sentimento da frase usando deep learning com TensorFlow. Este exemplo simples mostra como é possível criar um classificador de texto com poucas linhas de código! 🚀

O modelo que criamos consegue prever o sentimento de uma frase, distinguindo entre positivo e negativo, com base em padrões aprendidos durante o treinamento. Ele faz isso utilizando deep learning com TensorFlow, aproveitando representações numéricas (embeddings) e redes neurais para processar o texto de forma eficiente e precisa.

Apesar de ser um exemplo simples, ele demonstra como é possível construir um classificador de texto básico com apenas algumas linhas de código. Ferramentas como TensorFlow tornam acessível a implementação de algoritmos complexos de aprendizado profundo, permitindo que mesmo iniciantes comecem a explorar o mundo da inteligência artificial aplicada a texto.